<a href="https://colab.research.google.com/github/hpan270/MSA/blob/master/keras_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [35]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

batch_size = 128
num_classes = 10
epochs = 100
data_augmentation = False
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [36]:
json_str=model.to_json()
open("keras_cifar10_e100.json",'w').write('json_str')

8

In [0]:
from keras import callbacks

In [0]:
csvlogger=callbacks.CSVLogger('keras_cifar10_csvlogger_e100.csv',separator=',',append=False)
chkpt=callbacks.ModelCheckpoint('keras_cifar10_chkpt_e100.hdf5',verbose=1, save_best_only=True)

In [39]:
print('Not using data augmentation.')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2,
          callbacks=[csvlogger,chkpt],
          shuffle=True)

Not using data augmentation.
Train on 40000 samples, validate on 10000 samples
Epoch 1/100
40000/40000 [==============================] - 5s 133us/step - loss: 2.0334 - acc: 0.2506 - val_loss: 1.8240 - val_acc: 0.3571

Epoch 00001: val_loss improved from inf to 1.82395, saving model to keras_cifar10_chkpt_e100.hdf5
Epoch 2/100
40000/40000 [==============================] - 5s 115us/step - loss: 1.7682 - acc: 0.3634 - val_loss: 1.6768 - val_acc: 0.3992

Epoch 00002: val_loss improved from 1.82395 to 1.67677, saving model to keras_cifar10_chkpt_e100.hdf5
Epoch 3/100
40000/40000 [==============================] - 5s 114us/step - loss: 1.6546 - acc: 0.4017 - val_loss: 1.5912 - val_acc: 0.4347

Epoch 00003: val_loss improved from 1.67677 to 1.59121, saving model to keras_cifar10_chkpt_e100.hdf5
Epoch 4/100
40000/40000 [==============================] - 5s 115us/step - loss: 1.5801 - acc: 0.4288 - val_loss: 1.4988 - val_acc: 0.4665

Epoch 00004: val_loss improved from 1.59121 to 1.49883, sav

In [40]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 1s 79us/step
Test loss: 0.6969126290798188
Test accuracy: 0.7808


In [41]:
!ls -lht

total 20M
-rw-r--r-- 1 root root 5.6K Aug  6 00:02 keras_cifar10_csvlogger_e100.csv
-rw-r--r-- 1 root root 9.6M Aug  6 00:01 keras_cifar10_chkpt_e100.hdf5
-rw-r--r-- 1 root root    8 Aug  5 23:54 keras_cifar10_e100.json
-rw-r--r-- 1 root root 9.6M Aug  5 23:13 keras_cifar10_chkpt.hdf5
-rw-r--r-- 1 root root 1.8K Aug  5 23:13 keras_cifar10_csvlogger.csv
-rw-r--r-- 1 root root    8 Aug  5 23:10 keras_cifar10.json
drwxr-xr-x 1 root root 4.0K Aug  2 16:06 sample_data


In [0]:
from google.colab import files

files.download('keras_cifar10_chkpt_e100.hdf5')
files.download('keras_cifar10_e100.json')
files.download('keras_cifar10_csvlogger_e100.csv')